In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

from scipy.stats import norm
from scipy.stats import multivariate_normal

In [4]:
#load data into memory
data = pd.DataFrame(data = np.genfromtxt('sim3.dat', delimiter='  '), columns=['y','x1','x2','z'])

print data.mean()

y     50.500000
x1    10.500000
x2     0.577000
z      0.990078
dtype: float64
